In [91]:
import os
import tarfile
# import urllib
import pandas as pd
from zipfile import ZipFile as zp

ROOT = os.path.join(os.path.dirname(os.path.dirname(os.path.abspath(""))), "machine-learning")
RAW = os.path.join(os.path.join(ROOT, "datasets"), "titanic")
# DATASET = os.path.join(RAW, "titanic.csv")

print(RAW)
def unpackTar(path: str = RAW):
    target_p = os.path.join(path, "titanic.zip")
    print(target_p)
    if not os.path.exists(target_p):
        print("raw data does not exist")
        return
    titanic_path = tarfile.open(target_p)
    try:
        titanic_path.extractall(path=RAW, filter="fully_trusted")
    except Exception as e:
        print("unable to extract data")
        raise e
    titanic_path.close()

def unpackZip(path: str = RAW):
    target_p = os.path.join(path, "titanic.zip")
    with zp(target_p) as zf:
        zf.extractall(path=path)

unpackZip(RAW)
train = None
test = None
gender = None
dataframes = {"train": None, "test": None, "gender": None}

for f_name in dataframes.keys():
    file: str = None
    if f_name == "gender":
        file = "gender_submission"
    else:
        file = f_name
    p = os.path.join(RAW, file+".csv")
    try:
        if os.path.exists(p):
            dataframes[f_name] = pd.read_csv(p)
    except Exception as e:
        print(f"error in reading: {f_name}")

x:\E\Documents\Code\Repo\machine-learning\datasets\titanic


In [92]:
dataframes

{'train':      PassengerId  Survived  Pclass  \
 0              1         0       3   
 1              2         1       1   
 2              3         1       3   
 3              4         1       1   
 4              5         0       3   
 ..           ...       ...     ...   
 886          887         0       2   
 887          888         1       1   
 888          889         0       3   
 889          890         1       1   
 890          891         0       3   
 
                                                   Name     Sex   Age  SibSp  \
 0                              Braund, Mr. Owen Harris    male  22.0      1   
 1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
 2                               Heikkinen, Miss. Laina  female  26.0      0   
 3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
 4                             Allen, Mr. William Henry    male  35.0      0   
 ..                                      

In [93]:
df_tr: pd.DataFrame = dataframes["train"]
df_tst: pd.DataFrame = dataframes["test"]

In [94]:
df_tr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [95]:
df_tr.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [96]:
df_tst.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [97]:
target_tr = df_tr["Embarked"].copy()
target_tst = df_tst["Embarked"].copy()

In [98]:
df_tr.drop("Survived", axis=1, inplace=True)
for d in (df_tr, df_tst):
    d.drop("Name", axis=1, inplace=True)
    d.drop("Embarked", axis=1, inplace=True)
    if "Name" in d:
        raise Exception(f"Name still in dataframe")

In [99]:
def get_percentage_null(data: pd.DataFrame):
    for col in data.columns:
        if data[col].isnull().values.any():
            pct_null = (data[col].isnull().sum()/len(data[col])) * 100
            print(f"Column {col} [{data[col].dtype}] has {pct_null} % null values")

In [100]:
print("Training Data")
get_percentage_null(df_tr)
print("\nTest Data")
get_percentage_null(df_tst)

Training Data
Column Age [float64] has 19.865319865319865 % null values
Column Cabin [object] has 77.10437710437711 % null values

Test Data
Column Age [float64] has 20.574162679425836 % null values
Column Fare [float64] has 0.23923444976076555 % null values
Column Cabin [object] has 78.22966507177034 % null values


In [101]:
# impute categorical features using knn imputer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer

In [102]:
cat_col = [col for col in df_tr.columns if df_tr[col].dtype != "int64"]
num_col = [col for col in df_tr.columns if col not in cat_col]

In [103]:
cat_col

['Sex', 'Age', 'Ticket', 'Fare', 'Cabin']

In [104]:
num_col

['PassengerId', 'Pclass', 'SibSp', 'Parch']

In [105]:
transform_cat = ColumnTransformer(
    [
        ("cat_encoder", OneHotEncoder(sparse_output=False), cat_col)
    ]
)

impute_scale = Pipeline(
    [
        ("imputer", KNNImputer(n_neighbors=5)),
        ("scaler", StandardScaler()) 
    ]
)
                   
full_pipeline = Pipeline(
    [
        ("categoryTransformer",transform_cat),
        ("imputeAndScale",impute_scale)
    ]
)

In [106]:
print(type(df_tr))

<class 'pandas.core.frame.DataFrame'>


In [107]:
transformer = full_pipeline.fit(df_tr)

In [108]:
transformer

Pipeline(steps=[('categoryTransformer',
                 ColumnTransformer(transformers=[('cat_encoder',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['Sex', 'Age', 'Ticket',
                                                   'Fare', 'Cabin'])])),
                ('imputeAndScale',
                 Pipeline(steps=[('imputer', KNNImputer()),
                                 ('scaler', StandardScaler())]))])

In [109]:
prepared_tr_data = transformer.transform(df_tr)
prepared_tr_data

array([[-0.73769513,  0.73769513, -0.03352008, ..., -0.06715343,
        -0.03352008,  0.54492498],
       [ 1.35557354, -1.35557354, -0.03352008, ..., -0.06715343,
        -0.03352008, -1.835115  ],
       [ 1.35557354, -1.35557354, -0.03352008, ..., -0.06715343,
        -0.03352008,  0.54492498],
       ...,
       [ 1.35557354, -1.35557354, -0.03352008, ..., -0.06715343,
        -0.03352008,  0.54492498],
       [-0.73769513,  0.73769513, -0.03352008, ..., -0.06715343,
        -0.03352008, -1.835115  ],
       [-0.73769513,  0.73769513, -0.03352008, ..., -0.06715343,
        -0.03352008,  0.54492498]])

In [110]:
target_tr

0      S
1      C
2      S
3      S
4      S
      ..
886    S
887    S
888    S
889    C
890    Q
Name: Embarked, Length: 891, dtype: object

In [111]:
target_encoder = OneHotEncoder(sparse_output=False)
prepared_tr_target = target_encoder.fit_transform(pd.DataFrame(target_tr))

In [112]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

param_grid = [
    {"n_neighbors": [i for i in range(1, 10)]},
    {"weights": ["uniform", "distance"]}
]
grid_search = GridSearchCV(KNeighborsClassifier(), param_grid=param_grid)

In [113]:
grid_search.fit(prepared_tr_data, prepared_tr_target)

GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid=[{'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9]},
                         {'weights': ['uniform', 'distance']}])

In [114]:
grid_search.best_estimator_

KNeighborsClassifier(n_neighbors=1)

In [115]:
grid_search.best_params_

{'n_neighbors': 1}

In [117]:
prepared_tst_data = transformer.fit_transform(df_tst)
prediction = grid_search.predict(prepared_tst_data)

ValueError: X has 692 features, but KNeighborsClassifier is expecting 1168 features as input.